# Creating batch to test partial feedback setting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [5]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator as ba

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [6]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [7]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [69]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [93]:
time_horizon = 100
T = 10
targets = []
for i in range(2):
    targets = targets + util.gen_tar_with_len(5)
#targets = [1 for i in range(T)]
values = tuple((v, v) for v in targets)
#val_ls = [values[i][0] for i in range(T)]
#print(val_ls)
feedback_prob = [1 for i in range(T)]
observabilities = [1 for i in range(T)]
feedback_type = "mab"
distributions = []
for i in range(9):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
#distributions.append(tuple([1/T for i in range(T)]))
chosen_distr = tuple(util.gen_distr(T))
#chosen_distr = tuple([1/T for i in range(T)])
#chosen_distr = tuple([0.031] + [0.051 for i in range(19)])
print("chosen = ", chosen_distr)
#len(chosen_distr)

[(0.06006006006006005, 0.1301301301301301, 0.3333333333333333, 0.2262262262262262, 0.01101101101101101, 0.11711711711711711, 0.0, 0.07207207207207206, 0.04004004004004004, 0.01001001001001001), (0.04804804804804804, 0.2652652652652652, 0.10210210210210209, 0.2382382382382382, 0.004004004004004004, 0.025025025025025023, 0.09109109109109108, 0.18318318318318316, 0.04204204204204204, 0.001001001001001001), (0.031, 0.098, 0.015, 0.114, 0.078, 0.043, 0.347, 0.23, 0.033, 0.011), (0.022022022022022022, 0.026026026026026026, 0.06606606606606608, 0.056056056056056056, 0.03203203203203203, 0.041041041041041046, 0.01001001001001001, 0.01701701701701702, 0.5155155155155156, 0.21421421421421422), (0.26600000000000007, 0.06300000000000001, 0.17100000000000004, 0.09700000000000002, 0.06800000000000002, 0.013000000000000001, 0.036000000000000004, 0.09100000000000001, 0.11000000000000001, 0.08500000000000002), (0.076, 0.16, 0.181, 0.073, 0.225, 0.007, 0.047, 0.081, 0.067, 0.083), (0.06006006006006005, 

In [94]:
g = game.PartialFeedbackGame(values, time_horizon, observabilities, feedback_prob, feedback_type = "full")
g.attackers = [1]
g.defenders = [0]
d = [bm.FB(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
prof.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
prof.append(attackers.StackelbergAttacker(g, 1, 1))
for i in range(10):
    prof.append(attackers.SUQR(g,1))
#chosen_a_prof = (attackers.FrequentistUnknownStochasticAttacker(g, 1, 1))
#chosen_a_prof = attackers.SUQR(g,1)
#prof.append(attackers.FrequentistUnknownStochasticAttacker(g, 1, 1))
#a = [deepcopy(chosen_a_prof)]
#prof.append(chosen_a_prof)
a = [attackers.StochasticAttacker(g, 1, 1, *chosen_distr)]
#a = [attackers.StackelbergAttacker(g, 1, 1)]
#shuffle(prof)
#print("Real attacker is: ", a[0])
g.set_players(d, a, prof)

In [95]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(x) for x in range(len(targets))]) + "," + ",".join(["Feed_prob" + str(x) for x in range(len(targets))]) + ",Feed_type" + "\n" 

In [96]:
def print_row(targets, time_horizon, d, a, p, obs, fprob, ftype):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

In [97]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,6,7,8,9,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Obs0,Obs1,Obs2,Obs3,Obs4,Obs5,Obs6,Obs7,Obs8,Obs9,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_prob6,Feed_prob7,Feed_prob8,Feed_prob9,Feed_type\n'

In [98]:
defender =["FB1"]

In [99]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [100]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Experiments"
batch_name = "MAB_sto_10_tar"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [101]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type))

In [125]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,6,7,...,Feed_prob16,Feed_prob17,Feed_prob18,Feed_prob19,Feed_prob20,Feed_prob21,Feed_prob22,Feed_prob23,Feed_prob24,Feed_type
0,FB1_vs_sto1-0.04-0.04-0.04-0.04-0.04-0.04-0.04...,100,0.896,0.704,0.817,0.889,0.811,0.96,0.98,0.865,...,1,1,1,1,1,1,1,1,1,full


In [39]:
d = []
for i in range(45):
    d.append(tuple(util.gen_distr(T)))
pr = [attackers.StochasticAttacker(g, 1, 1, *x) for x in d]


['sto1-0.472-0.014-0.037-0.004-0.473', 'sto1-0.349-0.354-0.045-0.183-0.069', 'sto1-0.285-0.155-0.157-0.219-0.184', 'sto1-0.001-0.287-0.518-0.117-0.077', 'sto1-0.20300000000000004-0.5760000000000001-0.031000000000000003-0.10600000000000001-0.08400000000000002', 'sto1-0.167-0.324-0.051-0.262-0.196', 'sto1-0.058941058941058944-0.06793206793206795-0.04295704295704296-0.07392607392607393-0.7562437562437563', 'sto1-0.11411411411411411-0.11411411411411411-0.28828828828828823-0.4534534534534534-0.030030030030030026', 'sto1-0.029-0.113-0.708-0.124-0.026', 'sto1-0.32567432567432564-0.12987012987012986-0.1748251748251748-0.1928071928071928-0.1768231768231768', 'sto1-0.209-0.069-0.336-0.196-0.19', 'sto1-0.17082917082917087-0.007992007992007994-0.19180819180819184-0.48251748251748255-0.14685314685314685', 'sto1-0.13000000000000003-0.26000000000000006-0.3360000000000001-0.19600000000000004-0.07800000000000001', 'sto1-0.366-0.066-0.481-0.069-0.018', 'sto1-0.14185814185814183-0.06393606393606394-0.198

In [82]:
print([util.print_adv(k) for k in prof[:4]])

['sto1-0.008008008008008008-0.016016016016016016-0.08008008008008008-0.058058058058058054-0.02302302302302302-0.04304304304304304-0.035035035035035036-0.04104104104104104-0.10010010010010009-0.06706706706706707-0.0-0.006006006006006005-0.001001001001001001-0.13913913913913914-0.027027027027027025-0.14614614614614613-0.04104104104104104-0.015015015015015013-0.04004004004004004-0.01201201201201201-0.01201201201201201-0.035035035035035036-0.008008008008008008-0.02402402402402402-0.02202202202202202', 'sto1-0.058941058941058944-0.07692307692307693-0.007992007992007994-0.012987012987012988-0.03796203796203797-0.03596403596403597-0.04695304695304696-0.02197802197802198-0.03596403596403597-0.0019980019980019984-0.019980019980019983-0.014985014985014986-0.07992007992007993-0.016983016983016987-0.09690309690309691-0.029970029970029972-0.003996003996003997-0.0-0.09890109890109891-0.15284715284715286-0.0029970029970029974-0.02497502497502498-0.04995004995004996-0.06093906093906094-0.0089910089910

In [ ]:
print([])